In [1]:
import numpy as np
import copy
import dimod
import random
import math
from math import log
import dimod
import pprint
import neal
import numpy.linalg as LA #NEW!!!

Ignore the following blob, it just has the functions for building the rest of the code as usual

In [2]:
# v = our V matrix which is p x n matrix
# w = our W matrix which is p x k
# h = our H matrix which is k x n
# k = # of clusters

'''
    V        W        H
[v11, v12]  [w11, w12]  [h11, h12]
[v21, v22]  [w21, w22]  [h21, h22]
(v11 - (w11 * h11) + (w12 * h21) )^2 +
(v12 - (w11 * h12) + (w12 * h22) )^2 +
(v21 - (w21 * h11) + (w22 * h21) )^2 + 
(v22 - (w21 * h12) + (w22 * h22) )^2
'''

# Annealer will return binary value, need to convert back to reals
# using approximation
# Convert real value by taking floor of float value then make binary
def approx_real(r):
    return bin(math.floor(r).split('0b')[1])

def bin_to_real(binstr):
    return int(binstr,2)

# Figure out all v_ij - sum_h w_ih * h_hj
# pass in k , # of clusters

def find_vars(v,k):

    v_list = list()

    # Store our wh values and in reverse
    x_dict = {}
    x_dict_rev = {}

    # store our v values
    # This will essentially be a set of nested dictionaries
    '''
       (x, y) : { 
                   v_val: { v} , 
                   wh: { (wh_ik, wh_kj) }
                }
    '''
    # Dict for our V position and values in V
    v_dict = {}

    # 
    wh_dict = {}
 
    # Get correct dimensions
    # V is p x n 
    p = v.shape[0]
    n = v.shape[1]

    # W is p x k
    w_rows = p
    w_cols = k

    # H is k x n
    h_rows = k
    h_cols = n


    #Get the V's
    # V is p x n 
    for i in range(0,p):
        for j in range(0, n):
            # stringify what is in V at this location
            i_idx = i+1
            j_idx = j+1
            v_dict[i_idx,j_idx] = {}
            v_dict[i_idx,j_idx]['v_val'] = str(v[i][j])
            v_dict[i_idx,j_idx]['wh'] = []
            #v_str = str(v[i][j]) + "-"
            #v_list.append(v_str)

    # Build WH
    # WH will be same as V , so p x n
    wh_cnt = 1
    for i in range(0,w_rows):
        for j in range(0,h_cols):
            # This is just indexing to make it match and not index from 0
            i_idx = i+1
            j_idx = j+1
            for l in range(0,w_cols):   # This is the column vector selection
                #print("w" + str(i+1) + str(l+1) + "h" + str(l+1) + str(j+1))
                #x_dict['x'+str(wh_cnt)] = ("w" + str(i+1) + str(l+1) + "h" + str(l+1) + str(j+1))
                x_dict['x'+str(wh_cnt)] = ("w" + str(i+1) + str(l+1), "h" + str(l+1) + str(j+1))
                x_dict_rev[("w" + str(i+1) + str(l+1), "h" + str(l+1) + str(j+1))] = 'x'+str(wh_cnt)
                v_dict[i_idx,j_idx]['wh'].append( ("w"+str(i+1) + str(l+1), "h"+str(l+1)+str(j+1)) )
                wh_cnt += 1
               # x_dict['x'+str(i)] = "w" + str(i+1) + str(l+1) + "h" + str(l+1) + str(j+1)

    '''
    for k,v in x_dict.items():
        print(k,":", v)
    for k,v in x_dict_rev.items():
        print(k,":", v)
    for k,v in v_dict.items():
        print(k,":",v)
    '''
    return v_dict, x_dict, x_dict_rev, n
            

# Ax-b is similar to V-WH
# V = b here
# W = A
# H = x ??
## Code from Ajinkya Borle in nick_quadratic_automated

#Converts a ||Ax-b|| problem into a QUBO
def qubo_prep(A,b,n,bitspower,varnames=None):
    #Input:
    #'A' is the matrix which is a np.array(), a matrix which is m x n
    #b is the vector in the expression : Ax-b, an np.array() of length m
    #n is an integer value that is the length of the column vector x
    #bitspower is a list of powers that would be applied in a 2s complement way, 
    #eg : [2,1,0] would be applied with another bit tacked on front as : -2^(len(bitspower) + 2^(2) + 2^(1) + 2^(0)
    #varnames if provided is a list of variable name. eg : varnames = [x1,x2,x3]
    #Note : the implementation requires the list to be a set of decreasing and consecutive numbers, sorry!
    #Output:
    #A dictionary Qdict which is the QUBO for this problem
    #A dictionary Qdict_alt which the same QUBO but with alphanumeric variable (keys) names
    n_i_ctr = 0
    i_powerctr = 0
    i_twosymb = 0
    
    str_bitspower = ['null'] + [str(item) for item in bitspower]
    powersoftwo = np.zeros(len(bitspower)+1)
    
    for i in range(0,len(bitspower)+1):
        if i==0:
            powersoftwo[i] = (2**(bitspower[i]+1))*(-1)
        else:
            powersoftwo[i] = 2**(bitspower[i-1])
    Qinit = np.zeros([n,n]) #A proto-QUBO that preprocesses some values for the actual QUBO. Initial QUBO if you will
    Qdict = {} #A dictionary that stores the final QUBO
    
    Qdict_alt = {} #This dictionary saves the alternate QUBO with alphanumeric variable (keys) names
    index_dict = {} #Dictionary that maps alphanumeric variables to the integer ones
    
    for i in range(0,n):
        for j in range(i,n):
            Qinit[i,j] = 2*sum(A[:,i]*A[:,j])    
    bnew = 2*b
    
    
    for i in range(0,n*len(powersoftwo)):
        if i%len(powersoftwo)==0 and i>0:
            n_i_ctr = n_i_ctr + 1
            i_powerctr=0
        n_j_ctr = n_i_ctr
        j_powerctr = i_powerctr
        for j in range(i,n*len(powersoftwo)):
            if i==j:#Linear coefficient
                Qdict[i,i] = (powersoftwo[i_powerctr]**2)*(sum(A[:,n_i_ctr]**2)) - powersoftwo[i_powerctr]*sum(A[:,n_i_ctr]*bnew)
                if varnames != None:
                    tempvar1 = varnames[n_i_ctr] + '_' + str_bitspower[i_powerctr]
                    index_dict[tempvar1] = i
                    Qdict_alt[tempvar1,tempvar1] = Qdict[i,i]
            else:#Quadratic coefficient
                if j%len(powersoftwo)==0 and j>0:
                    n_j_ctr = n_j_ctr + 1
                    j_powerctr = 0
                Qdict[i,j] = powersoftwo[i_powerctr]*powersoftwo[j_powerctr]*Qinit[n_i_ctr,n_j_ctr]
                if varnames != None:
                    tempvar2 = varnames[n_j_ctr] + '_' + str_bitspower[j_powerctr]
                    Qdict_alt[tempvar1,tempvar2] = Qdict[i,j]
            
            j_powerctr = j_powerctr + 1
        i_powerctr = i_powerctr + 1
    
    if varnames != None:
        return Qdict, Qdict_alt, index_dict
    else:
        return Qdict #just return the bare bones if varnames not requested

#This is just to convert a dictionary based result into a binary string based result
def get_bin_str(config,isising=True):
    #Input:
    #config is a dictionary
    #isising is True if config has -1 or +1 and False if config has 0 or 1
    #Output:
    # a binary string of 0s and 1s
    binstr = ""
    if isising == True:
        for i in range(0,len(config)):
            if config[i] == 1:
                binstr += str(1)
            elif config[i] == -1:
                binstr += str(0)
    else:
        for i in range(0,len(config)):
            if config[i] == 1:
                binstr += str(1)
            elif config[i] == 0:
                binstr += str(0)
    return binstr

#processes the binary string into a np.array vector.
def qubo_to_real(binstr,n,prec_list):
    #prepare the powers_of_two list
    
    powers_of_two = []
    powers_of_two.append(-2**(prec_list[0]+1))
    
    
    for i in range(0,len(prec_list)):
        powers_of_two.append(2**(prec_list[i]))
    #Now the actual number
    bin_ctr=0
    cur_real = np.zeros(n)
    for i in range(0,n):
        for j in range(0,len(powers_of_two)):
            cur_real[i] += powers_of_two[j]*int(binstr[bin_ctr])
            bin_ctr += 1
    
    return cur_real

def qubo_prep_nonneg(A,b,n,bitspower, varnames=None):
    #Same as qubo_prep but only for non-negative values
    #bitspower = [0] for binary values
    
    
    n_i_ctr = 0
    i_powerctr = 0
    i_twosymb = 0
    
    str_bitspower =[str(item) for item in bitspower]
    powersoftwo = np.zeros(len(bitspower))
    
    for i in range(0,len(bitspower)):
        powersoftwo[i] = 2**(bitspower[i])
        
    Qinit = np.zeros([n,n])
    Qdict = {} #The dictionary for our qubo
    
    Qdict_alt = {} #This dictionary saves the alternate QUBO with alphanumeric variable (keys) names
    index_dict = {} #Dictionary that maps alphanumeric variables to the integer ones
    
    for i in range(0,n):
        for j in range(i,n):
            Qinit[i,j] = 2*sum(A[:,i]*A[:,j])    
    bnew = 2*b   
    
    for i in range(0,n*len(powersoftwo)):
        if i%len(powersoftwo)==0 and i>0:
            n_i_ctr = n_i_ctr + 1
            i_powerctr=0
        n_j_ctr = n_i_ctr
        j_powerctr = i_powerctr
        for j in range(i,n*len(powersoftwo)):
            if i==j: #Linear coefficient
                Qdict[i,i] = (powersoftwo[i_powerctr]**2)*(sum(A[:,n_i_ctr]**2)) - powersoftwo[i_powerctr]*sum(A[:,n_i_ctr]*bnew)
                if varnames != None:
                    tempvar1 = varnames[n_i_ctr] + '_' + str_bitspower[i_powerctr]
                    index_dict[tempvar1] = i
                    Qdict_alt[tempvar1,tempvar1] = Qdict[i,i]
            else: #Quadratic coefficient
                if j%len(powersoftwo)==0 and j>0:
                    n_j_ctr = n_j_ctr + 1
                    j_powerctr = 0
                Qdict[i,j] = powersoftwo[i_powerctr]*powersoftwo[j_powerctr]*Qinit[n_i_ctr,n_j_ctr]
                if varnames != None:
                    tempvar2 = varnames[n_j_ctr] + '_' + str_bitspower[j_powerctr]
                    Qdict_alt[tempvar1,tempvar2] = Qdict[i,j]
            
            j_powerctr = j_powerctr + 1
        i_powerctr = i_powerctr + 1
    
    if varnames != None:
        return Qdict, Qdict_alt, index_dict
    else:
        return Qdict #just return the bare bones if varnames not requested

#Function to convert the solution dictionary from alphanumeric variables to integer 
def convert_result(soln_dict,index):
    new_dict = {}
    for key,value in soln_dict.items():
        new_dict[index[key]] = value
    return new_dict

In the next cell is where the action lies

In [8]:
# In a 2x2 situation, we basicallay have to send one quadratic expression at a time
Q = {}
Q_alt = {}
index = {}
Q_total = {}


#NEW!!!! Here i am generating synthetic examples from which we shall create a v
#We shall then use a solver to then 'reverse engineer' W and H by passing a v
W_synth = np.random.randint(-4,4,size=(2,2))
H_synth = np.random.randint(0,2,size=(2,2))

v = np.matmul(W_synth,H_synth)#np.array([[3,2], [3,1]])
print("V is:\n",v)

prec_list = [1,0]
prec_list_str = ['null', '1', '0']


# v = our V matrix which is p x n matrix
# w = our W matrix which is p x k
# h = our H matrix which is k x n (binary)
# k = # of clusters


k = 2
A = np.zeros([1,k])
A += 1  # make matrix all 1's

v_rows = v.shape[0]
v_cols = v.shape[1]

v_dict, x_dict, x_dict_rev, n = find_vars(v,k)

for key, val in v_dict.items():
   #print(v_dict[key]['wh'])
    # Go through individual list of tuples
    varnames = []
    for item in v_dict[key]['wh']:
        # Get our corresponding x values to WH values
        varnames.append(x_dict_rev[item])
        # Build a row vector of 1's for A
        A = np.zeros([1,k])
        A += 1
        # Get each element of V for qubo_prep
        # print(varnames)
        # Also store them as a floating point number vs a string
    
    #NEW!!! the problem was here, you had to remove the loop above
    b = float(v_dict[key]['v_val'])
    #print(varnames)
    Q, Q_alt, index = qubo_prep(A,b,n,prec_list,varnames=varnames)
    # Put everything from each Q_alt dict into master Q_total
    for key, val in Q_alt.items():
        # Check if key is already here, if so add to it
        if key in Q_total:
            #print(key)
            Q_total[key] += val
        else:
            #print('zzaa')
            Q_total[key] = val


#print(Q_total)
#pprint.pprint(Q_total)

# linearization
# Delta == lagaragne param
delta = 5
for x_key, x_val in x_dict.items():
    temp_h = x_val[1]
    #print(temp_h)
    for prec in prec_list_str:
        temp_x = x_key + "_" + prec
        #print(test)
        temp_w = x_val[0] + "_" + prec
        #print(temp_w)
        Q_total[temp_w, temp_h] = 2 * delta
        Q_total[temp_x, temp_w] = -4 * delta
        Q_total[temp_x, temp_h] = -4 * delta
        Q_total[temp_x, temp_x] += 6 * delta

#pprint.pprint(Q_total)

sampler = neal.SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q_total, num_reads=40, num_sweeps=99999)
solution_dict = {}
solution_dict = sampleset.first.sample

p = v.shape[0]
n = v.shape[1]

W = np.zeros([p,k])
H = np.zeros([k,n])

for i in range(0,k):
    for j in range(0,n):
        temp_h = "h" + str(i+1) + str(j+1)
        H[i,j] = solution_dict[temp_h]
        
for i in range(0,p):
    for j in range(0,k):
        temp_w = "w" + str(i+1) + str(j+1)
        for sol_key, sol_val in solution_dict.items():
            if temp_w in sol_key:
                #print(temp_w, sol_key)
                temp_str = sol_key.split('_')[1]
                if temp_str == "null":
                    W[i,j] += -(2**(prec_list[0]+1))*sol_val
                else:
                    W[i,j] += (2**int(temp_str))*sol_val
print("W found:\n",W)
print("H found:\n",H)
print("Check WxH:\n",np.matmul(W,H))
print("QUBO Energy: ",sampleset.first.energy)

V is:
 [[ 1  0]
 [-1  0]]
W found:
 [[-4.  1.]
 [-3. -1.]]
H found:
 [[0. 0.]
 [1. 0.]]
Check WxH:
 [[ 1.  0.]
 [-1.  0.]]
QUBO Energy:  -2.0


Where V = WH, ||V-WH||=0 if answer is right

In [9]:
LA.norm(v - np.matmul(W,H))

0.0

Also, where V= WH, The best energy is -||V||^2 (for frobenius norm)

In [10]:
LA.norm(v)**2

2.0000000000000004